In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [4]:
#creates the training dataset
df_train = pd.read_csv("data/final/final_training.csv")
df_train = df_train[df_train['pot_doc'] != '[]']
X_train_df=pd.DataFrame(data = np.array(df_train), columns = df_train.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_train_df = pd.DataFrame(data = np.array(df_train['documentid']), columns = ['documentid'])

#creates the testing dataset
df_test = pd.read_csv("data/final/final_test.csv")
X_test_df=pd.DataFrame(data = np.array(df_test), columns = df_test.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_test_df = pd.DataFrame(data = np.array(df_test['documentid']), columns = ['documentid'])


In [5]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [6]:

def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 10)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 09:40:39,411] A new study created in memory with name: no-name-fa620f59-297e-4703-87ad-d6675bf568c7
[I 2023-01-29 09:40:39,790] Trial 0 finished with value: 0.1618122977346278 and parameters: {'x': 0.422449253134044, 'y': 2.7032373852476663, 'z': 3.7033117905206527}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:40:40,166] Trial 1 finished with value: 0.1618122977346278 and parameters: {'x': 6.823255934474085, 'y': 8.608406109492662, 'z': 6.823901041259494}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:40:40,540] Trial 2 finished with value: 0.1618122977346278 and parameters: {'x': 1.3643443004189049, 'y': 0.1049793659032161, 'z': 8.086628560654672}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:40:40,917] Trial 3 finished with value: 0.1618122977346278 and parameters: {'x': 4.922514669593223, 'y': 2.1890139632510754, 'z': 7.721879274046026}. Best is trial 0 with value: 0.1618122977346278.
[I 2023-01-29 09:40:41,29

KeyboardInterrupt: 

In [7]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])


In [8]:
'''
calculates the percentage where our predicted data conforms to our acutal data
'''
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
    
#from our model, we have a predicted accuracy of 89.51612903225806
print(calculate_percentage(res,real))

89.51612903225806
